In [18]:
from bs4 import BeautifulSoup
import numpy
import requests as res
from urllib.parse import urlparse, parse_qs, urljoin
import json
import openpyxl
import time
import random

In [19]:
random.seed(1287)

In [20]:
# daily webtoon

def crawler_daily_webtoon():
    get_daily_webtoon=res.get("http://comic.naver.com/webtoon/weekday.nhn")
    soup=BeautifulSoup(get_daily_webtoon.content,"lxml")
    tag=soup.select(".daily_all a.title")
    webtoon_list=[{"title":a.get("title"),"link":str("http://comic.naver.com")+a.get("href")} for a in tag]
    
    return webtoon_list

    
    

In [21]:
# all_episode

def cralwer_episode(webtoon):
    target_webtoon=list()
    page_num=1
    is_unlast=True
    while is_unlast:
        link=webtoon["link"]+"&page="+str(page_num)
        get_page=res.get(link)
        page=BeautifulSoup(get_page.content,"lxml")
        a_tags=page.select(".viewList td.title a")
        for a_tag in a_tags:
            episode=a_tag.text.replace("\n","").replace("\r","").replace("\t","")
            episode_url="http://comic.naver.com"+str(a_tag.get("href"))
            
            if episode_url not in target_webtoon:
                target_webtoon.append(episode_url)
                
            else:
                is_unlast=False
                
        page_num+=1
    return target_webtoon



In [22]:
def get_comments(soup,url):
        title= soup.title.text.split(':')[0]
        titleId =str(parse_qs(urlparse(url).query)['titleId'][0]) #웹툰 고유아이디값
        no = str(parse_qs(urlparse(url).query)['no'][0])# 웹툰 그 회차 페이지

        comment_url = 'http://comic.naver.com' + '/comment/comment.nhn?titleId=' + titleId + '&no=' + no
        objectId = titleId + '_' + no
        sub_title=soup.select(".tit_area h3")
        sub_title=sub_title[0].text
        page_count = 1

        u = 'http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery1113012327623800394427_1489937311100&lang=ko&country=KR&objectId=' +objectId+ '&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&sort=NEW&_=1489937311112'

        comment_list=list()

        while True:
            comment_url=u+"&page="+str(page_count)
            header = {
                "Host": "apis.naver.com",
                "Referer": "http://comic.naver.com/comment/comment.nhn?titleId=" + titleId + "&no=" + no,
                "Content-Type": "application/javascript"
            }

            rq = res.get(comment_url, headers = header)
            soup = BeautifulSoup(rq.content, 'lxml')
            
           
            try:
                content_text = soup.select('p')[0].text
                one = content_text.find('(') + 1
                two = content_text.find(');')
                content = json.loads(content_text[one:two])

                comments = content['result']['commentList']



                for comment in comments:
                    comment_list.append({
                            "sub_title":sub_title,
                            "title": title,
                            "comment":comment["contents"],
                            "sympathy_count":comment["sympathyCount"],
                             "episode_num":no})
#                     print(comment["contents"])
                if not len(comments):
                    return comment_list
                    break
                    
                else:
                    page_count += 1
            except:
                pass

In [23]:
a=crawler_daily_webtoon()

In [24]:
for ind in a:
    if ind["title"]=="열정호구":
        print(a.index(ind))

46


In [25]:
a[46]

{'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=686312&weekday=tue',
 'title': '열정호구'}

In [26]:
fire_hogu=cralwer_episode(a[46])

In [27]:
fire_hogu

['http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=64&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=63&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=62&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=61&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=60&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=59&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=58&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=57&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=56&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=55&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=54&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=686312&no=53&weekday=tue',
 'http://comic.naver.com/webtoon/detail.nhn?titleId=

In [53]:
import queue
import threading


all_comments=[]
input_queue=queue.Queue()
out_queue=queue.Queue()

class Input_Thread(threading.Thread):
    def __init__(self,input_queue,out_queue):
        
        threading.Thread.__init__(self)
        self.input_queue=input_queue
        self.out_queue=out_queue
    
    def run(self):
        while True:
            url=self.input_queue.get()
#             rt=res.get(url)
#             soup=BeautifulSoup(rt.content,"lxml")
            self.out_queue.put(url)
            
            
            self.input_queue.task_done()



class Output_Thread(threading.Thread):
    def __init__(self,out_queue):
        
        threading.Thread.__init__(self)
        self.out_queue=out_queue
    
    def run(self):
        while True:
            data=self.out_queue.get()
            rt=res.get(data)
            soup=BeautifulSoup(rt.content,"lxml")
            time.sleep(random.uniform(2.1,2.8))
            get_comments(soup,data)
            
            self.out_queue.task_done()


            
            

In [ ]:
fire_hogu[-2:]

In [ ]:

for i in fire_hogu[-2:]:
    th=Input_Thread(input_queue,out_queue)
    th.setDaemon(True)
    th.start()
    
for epi in fire_hogu[-2:]:
    input_queue.put(epi)

for epi1 in range(2):
    out=Output_Thread(out_queue)
    out.setDaemon(True)
    out.start()
    
input_queue.join()
out_queue.join()

In [ ]:
print(1)

In [ ]:
def multi_crawl(webtoon_episode_list,que):
    
    if int(len(webtoon_episode_list)%5)!=0: #나머지가 존재한다면
            unit=int(len(webtoon_episode_list)//5)
            surplus=int(len(webtoon_episode_list)%5)
            
            # 몫으로 배분된 웹툰 페이지 크롤링
            for i in range(int(len(webtoon_episode_list)/5)):
                a=[]
                a=webtoon_episode_list[i*5:i*5+5]
                thread_list=[]
                for url in a:
                    rt=res.get(url)
                    soup=BeautifulSoup(rt.content,"lxml")
                    worker=Thread(target=get_comments,args=(soup,url))
                    worker.setDaemon(True)
                    thread_list.append(worker)
                    worker.start()
                
                for end in thread_list:
                    end.join()
                    
                    
            # 나머지 페이지 크롤링
            thread_list=[]
            for url in webtoon_episode_list[-surplus:]:
                rt=res.get(url)
                soup=BeautifulSoup(rt.content,"lxml")
                worker=Thread(target=get_comments,args=(soup,url))
                worker.setDaemon(True)
                thread_list.append(worker)
                worker.start()

            for end in thread_list:
                end.join()
                
                
            return que

    else:
           for i in range(int(len(webtoon_episode_list)/5)):
                a=[]
                a=webtoon_episode_list[i*5:i*5+5]
                thread_list=[]
                for url in a:
                    rt=res.get(url)
                    soup=BeautifulSoup(rt.content,"lxml")
                    worker=Thread(target=get_comments,args=(soup,url,que))
                    worker.setDaemon(True)
                    thread_list.append(worker)
                    worker.start()
                
                for end in thread_list:
                    end.join()
                    
           return que
                 

In [ ]:
import queue
from threading import Thread
que=queue.Queue()
multi_crawl(fire_hogu[:12],que)

In [ ]:
import queue
from threading import Thread

In [ ]:
fire_hogu[-1:]

In [ ]:


all_comments=list()
for episode in fire_hogu[-1:]:
    
    rt=res.get(episode)
    soup=BeautifulSoup(rt.content,"lxml")
    
    all_comments.append(get_comments(soup,episode))
    time.sleep(random.uniform(2.0,3.1))

In [ ]:
print((random.uniform(1.2,1.8)))

In [ ]:
all_comments

In [ ]:
from multiprocessing import Pool



# Queue to Dataframe
---

In [ ]:
import pandas as pd

In [ ]:
data=pd.DataFrame(all_comments[0])

In [ ]:
data1=pd.DataFrame(all_comments[1])

merge=pd.concat([data,data1],ignore_index=True)
len(merge)

In [ ]:
first=pd.DataFrame()
for one_episode in all_comments:
    data=pd.DataFrame(one_episode)
    first=pd.concat([first,data],ignore_index=True)
    
    

In [ ]:
len(first)

In [ ]:
writer=pd.ExcelWriter("outpu_final.xlsx")
writer=pd.

v first.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
df_1=data.groupby(["comment","sympathy_count"])

In [ ]:
exact_large=first.nlargest(1000,"sympathy_count")

In [ ]:
writer1=pd.ExcelWriter("greateest_count.xlsx")
exact_large.to_excel(writer1,"Sheet1")
writer1.save()

In [ ]:
import threading, queue 
import time 


class Catcher(threading.Thread): 
    # Thread Extends 
    def __init__(self, id, dataQueue): 
        threading.Thread.__init__(self) 
        self.id = id 
        # for print 
        self.dataQueue = dataQueue # data queue 
    def run(self): 
        while True:	# thread task 
            data = self.dataQueue.get()	# queue element get 
            print("[" + str(self.id) + " : " + str(data) + "]") 
            self.dataQueue.task_done()	# work done notify to queue 
def main(): 
    StringQueue = queue.Queue()	# create queue 
    
    for i in range(10000):
        StringQueue.put(i)	# put data 
    
    cat1 = Catcher("A", StringQueue)	# declare thread obj 
    cat1.setDaemon(True) 
    cat2 = Catcher("B", StringQueue) 
    cat2.setDaemon(True)
    cat3 = Catcher("C", StringQueue) 
    cat3.setDaemon(True)
    
    cat1.start()	# thread start 
    cat2.start() 
    cat3.start()
    StringQueue.put(3)
    StringQueue.put(4) 
    
     
if __name__ == "__main__": main()

# 출처: http://hurderella.tistory.com/86 [Hurderella]

In [50]:
u = 'http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery1113012327623800394427_1489937311100&lang=ko&country=KR&objectId=686312_1&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&sort=NEW&_=1489937311112'
# objectId = titleId + '_' + no
#         /comment_list = list()

#         while True:
comment_url = u + "&page=" + "5"
header = {
"Host": "apis.naver.com",
"Referer": "http://comic.naver.com/comment/comment.nhn?titleId=" +"686312"+ "&no=" +"1",
"Content-Type": "application/javascript"
}
# time.sleep(5)
rq = res.get(comment_url, headers=header)

soup = BeautifulSoup(rq.content, 'lxml')


In [51]:
soup

<html><body><p>jQuery1113012327623800394427_1489937311100({"success":true,"code":"1000","message":"ìš”ì²­ì</p></body></html>

In [52]:

content_text = soup.select('p')[0].text
one = content_text.find('(') + 1
two = content_text.find(');')
content = json.loads(content_text[one:two])

comments = content['result']['commentList']


comment_list=[]
for comment in comments:
    comment_list.append({
#             "sub_title":sub_title,
#             "title": title,
            "comment":comment["contents"],
            "sympathy_count":comment["sympathyCount"],})
#              "episode_num":no})

JSONDecodeError: Unterminated string starting at: line 1 column 41 (char 40)

In [33]:
comment_list

[{'comment': '자야 되는데 정주행할께 생겼다', 'sympathy_count': 0},
 {'comment': '그림체는 아름답지만 내용은 그렇지 않다..', 'sympathy_count': 10},
 {'comment': '처음엔 다들 그렇게 시작하지 ㅋㅋㅋㅋ', 'sympathy_count': 1},
 {'comment': '잘했다', 'sympathy_count': 0},
 {'comment': '아주 잉여인긴이야 잉여인간 잉여인간이 뭔줄 알어? 인간 떨거지 되는거야 이쌔77ㅣ야.',
  'sympathy_count': 1},
 {'comment': '헐... 내 얘기인가? ㅋㅋ', 'sympathy_count': 0},
 {'comment': '전 전국 노예자랑 편이 제일 맘에 듬 ㅎㅎ', 'sympathy_count': 2},
 {'comment': '아 진짜 찡해서 못보겠다 .. ( 항암제 준비', 'sympathy_count': 3},
 {'comment': '현실세계', 'sympathy_count': 2},
 {'comment': '처음에 편집장 뒷얼굴 볼때 묘하게 기분안좋았다\n이때부터 시작이였구나 편집장아',
  'sympathy_count': 3},
 {'comment': '부모님 해도 너무하시네...', 'sympathy_count': 3},
 {'comment': '취업만 하면 다 끝날 줄 알았는데 그게 아님ㅠㅜ', 'sympathy_count': 3},
 {'comment': '다음 날 바로 출근하라는 회사가 제일 위험한 회사다.', 'sympathy_count': 2},
 {'comment': "난 이제 20살인데 tv나 매체에 나오는 내 또래천재, 성공한애들이랑 늘 비교함 지 명문대 출신하나로 나를 한심하다 늘 그럼. 오늘도 학교다니며 과제하고 피곤해서 공강하루 누워쉬었는데 억지로 불키고 깨우면서 '게으른인생~~ 에휴~~' 라 가족앞에서 소리치고... 나 진짜 열심히 살려 노력하는데...",
  'sympathy_cou